<a href="https://colab.research.google.com/github/Tarunsaip/Deep_Learning_NN/blob/master/SimpleRNN_(Keras).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

In [0]:
all_chars = '0123456789+'

In [0]:
num_features = len(all_chars)

char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))

print('Number of features:', num_features)

Number of features: 11


In [0]:
def generate_data():
    first_num = np.random.randint(low=0,high=100)
    second_num = np.random.randint(low=0,high=100)
    example = str(first_num) + '+' + str(second_num)
    label = str(first_num+second_num)
    return example, label

generate_data()

('64+35', '99')

In [0]:
hidden_units = 128
max_time_steps = 5

model = Sequential([
    SimpleRNN(hidden_units, input_shape=(None, num_features)),
    RepeatVector(max_time_steps),
    SimpleRNN(hidden_units, return_sequences=True),
    TimeDistributed(Dense(num_features, activation='softmax'))
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 128)               17920     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [0]:
def vectorize_example(example, label):
    
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)
    
    for i, c in enumerate(example):
        x[diff_x+i, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
    for i, c in enumerate(label):
        y[diff_y+i, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
        
    return x, y

e, l = generate_data()
print('Text Example and Label:', e, l)
x, y = vectorize_example(e, l)
print('Vectorized Example and Label Shapes:', x.shape, y.shape)

Text Example and Label: 74+54 128
Vectorized Example and Label Shapes: (5, 11) (5, 11)


In [0]:
def devectorize_example(example):
    result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
    return ''.join(result)

devectorize_example(x)

'74+54'

In [0]:
devectorize_example(y)

'00128'

In [0]:
def create_dataset(num_examples=2000):

    x = np.zeros((num_examples, max_time_steps, num_features))
    y = np.zeros((num_examples, max_time_steps, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        e_v, l_v = vectorize_example(e, l)
        x[i] = e_v
        y[i] = l_v
    
    return x, y

x, y = create_dataset()
print(x.shape, y.shape)

(2000, 5, 11) (2000, 5, 11)


In [0]:
devectorize_example(x[0])

'91+65'

In [0]:
devectorize_example(y[0])

'00156'

In [0]:
l_cb = LambdaCallback()

es_cb = EarlyStopping(monitor = 'val_loss', patience = 10)
model.fit(x,y,epochs = 500,batch_size = 256,validation_split=0.2,verbose = False , callbacks = [es_cb,l_cb])



In [0]:
x_test , y_test = create_dataset(10)
preds = model.predict(x_test)
for i, pred in enumerate(preds) :
	y = devectorize_example(y_test[i])
	y_hat = devectorize_example(pred)
	col = 'green'
	if y!= y_hat:
		col = 'red'
	out = 'Input : '+ devectorize_example(x_test[i])+ ' Out : '+y+' Pred: '+y_hat
	print(colored(out,col))

Input : 79+58 Out : 00137 Pred: 00137
Input : 020+9 Out : 00029 Pred: 00032
Input : 69+14 Out : 00083 Pred: 00083
Input : 49+63 Out : 00112 Pred: 00112
Input : 61+66 Out : 00127 Pred: 00127
Input : 26+34 Out : 00060 Pred: 00060
Input : 51+30 Out : 00081 Pred: 00081
Input : 43+15 Out : 00058 Pred: 00058
Input : 25+53 Out : 00078 Pred: 00078
Input : 21+58 Out : 00079 Pred: 00089
